In [100]:
import pandas as pd
import numpy as np
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler,  LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Markdown,display
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [101]:
train_data = pd.read_excel("https://github.com/FlipRoboTechnologies/ML-Datasets/raw/main/Restaurant%20Food%20Cost/Data_Train.xlsx")

In [102]:
train_data

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300
...,...,...,...,...,...,...,...,...,...
12685,QUICK BITES,13228,"North Indian, Burger, Kebab","12noon – 12midnight (Mon, Tue, Wed, Thu, Sun)...",Hyderabad,Gachibowli,3.8,546 votes,500
12686,"CASUAL DINING,BAR",9686,"Goan, Continental","12noon – 1am (Mon-Fri),11am – 5pm, 7pm – 1am...",Mumbai,Bandra Kurla Complex,4.3,1214 votes,1800
12687,LOUNGE,11133,"Finger Food, Continental, Asian, Chinese",12noon – 12:30AM (Mon-Sun),Navi Mumbai,Vashi,4.0,608 votes,1300
12688,CASUAL DINING,6134,"North Indian, South Indian, Chinese, Street Food",6am – 10:45pm (Mon-Sun),Chennai,Maduravoyal,3.5,32 votes,400


In [103]:
test_data = pd.read_excel("https://github.com/FlipRoboTechnologies/ML-Datasets/raw/main/Restaurant%20Food%20Cost/Data_Test.xlsx")

In [104]:
test_data

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes
2,CASUAL DINING,1411,"North Indian, Seafood, Biryani, Chinese",11am – 11:30pm (Mon-Sun),Mumbai,Marine Lines,3.8,350 votes
3,None,204,Biryani,"9am – 10pm (Mon, Wed, Thu, Fri, Sat, Sun), 10:...",Faridabad,NIT,3.8,1445 votes
4,QUICK BITES,13453,"South Indian, Kerala",11am – 10pm (Mon-Sun),Kochi,Kaloor,3.6,23 votes
...,...,...,...,...,...,...,...,...
4226,CASUAL DINING,9057,"North Indian, Mughlai, Chinese",11:30am – 11:30pm (Mon-Sun),New Delhi,Punjabi Bagh,3.9,287 votes
4227,None,1247,"Biryani, North Indian, Sandwich, Salad, Wraps",11am – 1am (Mon-Sun),Bangalore,HSR Layout,4.3,469 votes
4228,QUICK BITES,8617,"Continental, North Indian",9:30am – 10:30pm (Mon-Sun),Faridabad,Sector 86,3.7,53 votes
4229,QUICK BITES,6485,"Rolls, Beverages","11am – 11:30pm (Mon, Tue, Wed, Thu, Sat, Sun),...",Kochi,Kochi,-,NaN


In [105]:
train_data.isna().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

In [106]:
test_data.isna().sum()

TITLE              0
RESTAURANT_ID      0
CUISINES           0
TIME               0
CITY              35
LOCALITY          30
RATING             2
VOTES            402
dtype: int64

In [107]:
train_data['VOTES'] = pd.to_numeric(train_data['VOTES'], errors='coerce')

# Handle non-numeric values in 'RATING' column
train_data['RATING'] = pd.to_numeric(train_data['RATING'], errors='coerce')

# Fill missing values for numerical columns with the mean
train_data['RATING'].fillna(train_data['RATING'].mean(), inplace=True)
train_data['VOTES'].fillna(train_data['VOTES'].mean(), inplace=True)

# Fill missing values for categorical columns with the mode
train_data['CITY'].fillna(train_data['CITY'].mode()[0], inplace=True)
train_data['LOCALITY'].fillna(train_data['LOCALITY'].mode()[0], inplace=True)

# Check if there are any remaining missing values
print(train_data.isnull().sum())

TITLE                0
RESTAURANT_ID        0
CUISINES             0
TIME                 0
CITY                 0
LOCALITY             0
RATING               0
VOTES            12690
COST                 0
dtype: int64


In [109]:
train_data['VOTES'] = pd.to_numeric(train_data['VOTES'], errors='coerce')

# Handle non-numeric values in 'RATING' column
train_data['RATING'] = pd.to_numeric(train_data['RATING'], errors='coerce')

# Fill missing values for numerical columns with the mean
train_data['RATING'].fillna(train_data['RATING'].mean(), inplace=True)
train_data['VOTES'].fillna(train_data['VOTES'].mean(), inplace=True)

# Fill missing values for categorical columns with the mode
train_data['CITY'].fillna(train_data['CITY'].mode()[0], inplace=True)
train_data['LOCALITY'].fillna(train_data['LOCALITY'].mode()[0], inplace=True)

# Check if there are any remaining missing values
print(train_data.isnull().sum())

TITLE            0
RESTAURANT_ID    0
CUISINES         0
TIME             0
CITY             0
LOCALITY         0
RATING           0
VOTES            0
COST             0
dtype: int64


In [108]:
train_data['VOTES'] = pd.to_numeric(train_data['VOTES'], errors='coerce')

# Fill missing values for numerical columns with a placeholder value
train_data['VOTES'].fillna(-1, inplace=True)

# Replace placeholder values with the mean
train_data['VOTES'].replace(-1, train_data['VOTES'].mean(), inplace=True)

In [110]:
test_data['VOTES'] = pd.to_numeric(test_data['VOTES'], errors='coerce')

# Fill missing values for numerical columns with a placeholder value
test_data['VOTES'].fillna(-1, inplace=True)

# Replace placeholder values with the mean
test_data['VOTES'].replace(-1, test_data['VOTES'].mean(), inplace=True)

# Fill missing values for categorical columns with the mode
test_data['CITY'].fillna(test_data['CITY'].mode()[0], inplace=True)
test_data['LOCALITY'].fillna(test_data['LOCALITY'].mode()[0], inplace=True)

# Check if there are any remaining missing values
print("\nMissing values after cleaning:")
print(test_data.isnull().sum())


Missing values after cleaning:
TITLE            0
RESTAURANT_ID    0
CUISINES         0
TIME             0
CITY             0
LOCALITY         0
RATING           2
VOTES            0
dtype: int64


In [111]:
test_data['RATING'] = pd.to_numeric(test_data['RATING'], errors='coerce')

# Fill missing values for numerical columns with the mean
test_data['RATING'].fillna(test_data['RATING'].mean(), inplace=True)

# Check if there are any remaining missing values
print("\nMissing values after filling:")
print(test_data.isnull().sum())


Missing values after filling:
TITLE            0
RESTAURANT_ID    0
CUISINES         0
TIME             0
CITY             0
LOCALITY         0
RATING           0
VOTES            0
dtype: int64


In [112]:
X_train = train_data.drop(columns=['COST'])  # Features
y_train = train_data['COST']

In [113]:
model = LinearRegression()

In [115]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression

# Define preprocessing steps for numerical and categorical features
numeric_features = ['RATING', 'VOTES']  # Adjust this based on your numerical columns
categorical_features = ['TITLE', 'CUISINES', 'CITY', 'LOCALITY']  # Adjust this based on your categorical columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with mean
    ('scaler', StandardScaler())  # Scale features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values with mode
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Combine preprocessing steps for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Append linear regression model to preprocessing pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# Fit the model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['RATING', 'VOTES']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['TITLE', 'CUISINES', 'CITY',
                                                   'LOCALITY'])])),
                ('regressor', LinearRegression())])

In [117]:
X_test=test_data

In [119]:
y_pred = model.predict(X_test)

In [120]:
y_pred


array([ 836.03552092,  568.01416301,  692.77907127, ..., 1429.05415609,
        316.47653341,  872.48105387])